<a href="https://colab.research.google.com/github/santanaemmanuel/redes_neurais_CESAR_SCHOOL/blob/main/Redes_Neurais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aluno: Emmanuel Dantas Ribeiro de Santana

#Data Preprocessing
Esta primeira etapa tem como objetivo realizar o download do dataset e prepará-lo em um grupo de treino e outro de testes. O conjunto de dados encontra-se no link do github a ser clonado

In [1]:
!git clone https://github.com/santanaemmanuel/mtg-cards.git

fatal: destination path 'mtg-cards' already exists and is not an empty directory.


Uma vez que o dataset foi carregado no notebook, o próximo passo será dividí-lo em grupos de treino e teste. Cada categoria possui 600 imagens, dais quais 120 foram movidas para a pasta de validação e 480 para treino.

In [2]:
import os
import shutil
import re

dataset_dir = "/content/mtg-cards"
base_dir = "mtg_dataset"

# Moves all training cat images to cats folder, training dog images to dogs folder
def val_train_maker(name, val_size):
  try:
      path_train = f"{base_dir}/train/{name}"
      path_val = f"{base_dir}/val/{name}"
      os.makedirs(path_train, exist_ok=True)
      os.makedirs(path_val, exist_ok=True)
  except OSError:
      print ("Creation of the directory failed")
  else:
      print ("Successfully created the directories")
  files_path = os.path.join(dataset_dir, name)
  files = os.listdir(files_path)
  file_count = 0
  for f in files:
    if file_count < val_size:
      shutil.move(f'{files_path}/{f}', path_train)
      file_count += 1
    else:
      shutil.move(f'{files_path}/{f}', path_val)
      file_count += 1

  print(f'Train set contains: {len(os.listdir(path_train))} files')
  print(f'Val set contains: {len(os.listdir(path_val))} files')

class_names = ['W','B','R','G','U']
for name in class_names:
   val_train_maker(name, 120)

Successfully created the directories
Train set contains: 120 files
Val set contains: 480 files
Successfully created the directories
Train set contains: 120 files
Val set contains: 480 files
Successfully created the directories
Train set contains: 120 files
Val set contains: 480 files
Successfully created the directories
Train set contains: 120 files
Val set contains: 480 files
Successfully created the directories
Train set contains: 120 files
Val set contains: 480 files


#Loading Data
Nessa etapa será feita a proparação dos dados para serem alimentados no modelo

In [3]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import copy

Transformações aléatorias serão aplicadas aos dados para criar artificialmente um numero maior de imagens para o modelo

In [4]:
# Make transforms and use data loaders

# We'll use these a lot, so make them variables
mean_nums = [0.485, 0.456, 0.406]
std_nums = [0.229, 0.224, 0.225]

chosen_transforms = {'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean_nums, std_nums)
]), 'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean_nums, std_nums)
]),
}

# Set the directory for the data
data_dir = 'mtg_dataset/'
# Use the image folder function to create datasets
chosen_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), chosen_transforms[x]) for x in ['train', 'val']}

Finalmente sao criados os elementos iteraveis com os dados

In [6]:
# Make iterables with the dataloaders
dataloaders = {x: torch.utils.data.DataLoader(chosen_datasets[x], batch_size=4,
  shuffle=True, num_workers=4)
              for x in ['train', 'val']}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [7]:
dataset_sizes = {x: len(chosen_datasets[x]) for x in ['train', 'val']}
class_names = chosen_datasets['train'].classes

In [8]:
print(dataset_sizes)
print(class_names)

{'train': 600, 'val': 2400}
['B', 'G', 'R', 'U', 'W']


#Modelo
O modelo resnet utilizado será o não treinado, uma vez que o problema de classifcação de cartas de magic soa bastante distante do treinamento inicial da rede.

In [9]:
print(
    torch.cuda.is_available(),
    torch.cuda.current_device(),
    torch.cuda.get_device_name(0),
    sep='\n'
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

True
0
Tesla K80


Será utilizada uma camada com 5 saídas no fim da rede, que deverão corresponder as categorias do dataset

In [10]:
# Setting up the model
# load in pretrained and reset final fully connected
res_mod = models.resnet34(pretrained=False)
num_ftrs = res_mod.fc.in_features
res_mod.fc = nn.Linear(num_ftrs, 5)

Nesta etapa são setados alguns dos hyperparâmetros da rede, como taca de aprendizado

In [11]:
res_mod = res_mod.to(device)
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(res_mod.parameters(), lr=0.0001, momentum=0.7)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Rotina de treino da rede

In [12]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            current_loss = 0.0
            current_corrects = 0

            # Here's where the training happens
            print('Iterating through data...')

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # We need to zero the gradients, don't forget it
                optimizer.zero_grad()

                # Time to carry out the forward training poss
                # We only need to log the loss stats if we are in training phase
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # We want variables to hold the loss statistics
                current_loss += loss.item() * inputs.size(0)
                current_corrects += torch.sum(preds == labels.data)

            epoch_loss = current_loss / dataset_sizes[phase]
            epoch_acc = current_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # Make a copy of the model if the accuracy on the validation set has improved
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_since = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_since // 60, time_since % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # Now we'll load in the best model weights and return it
    model.load_state_dict(best_model_wts)
    return model

In [13]:
# Visualize some images
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([mean_nums])
    std = np.array([std_nums])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # Pause a bit so that plots are updated

In [14]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_handeled = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_handeled += 1
                ax = plt.subplot(num_images//2, 2, images_handeled)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_handeled == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
base_model = train_model(res_mod, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=20)
visualize_model(base_model)
plt.show()

#Conclusões
A rede resnet não se mostrou muito eficiente na classificação de cartas de Magic nas circunstâncias aqui apresentadas. 

Durante o processo de treinamento, foram testados diferentes "lr" e momentos alem de uma maior quantidade de epocas.

O modelo estabilizou por volta de 14 a 16 epocas a depender dos parâmetros empregados.

Uma possibilidade que poderá ser explorada é a de aumentar a quantidade de elementos do dataset
.
